In [123]:
import bz2
import os

In [124]:
file = 'pagecounts/2018/2018-06/pagecounts-2018-06-01.bz2'
folder,filename = os.path.split(file)

In [11]:
data = bz2.open(file)

In [127]:
date = filename[11:-4]
print(date)

2018-06-01


In [12]:
data.peek(5000)

b"# Wikimedia page request counts for 01/06/2018 (dd/mm/yyyy) \n#\n# Each line shows 'project page daily-total hourly-counts'\n#\n# Project is 'language-code project-code'\n#\n# Project-code is\n#\n# b:wikibooks,\n# k:wiktionary,\n# n:wikinews,\n# q:wikiquote,\n# s:wikisource,\n# v:wikiversity,\n# wo:wikivoyage,\n# z:wikipedia (z added by merge script: wikipedia happens to be sorted last in dammit.lt files, but without suffix)\n#\n# Counts format: only hours with page view count > 0 (or data missing) are represented,\n#\n# Hour 0..23 shown as A..X (saves up to 22 bytes per line compared to comma separated values), followed by view count.\n# If data are missing for some hour (file missing or corrupt) a question mark (?) is shown,\n# and a adjusted daily total is extrapolated as follows: for each missing hour the total is incremented with hourly average\n#\n# Page titles are shown unmodified (preserves sort sequence)\n#\naa.b Main_Page 16 E1H1J1K2L3N1R1T1U3V1W1\naa.b User:EVula 34 G2J4K1

In [9]:
import pandas
import codecs
import gzip

In [36]:
import re
linefilter = re.compile("(\S+)")

In [40]:
wrapper = codecs.getreader('utf-8')
with bz2.open(filename, 'rb') as f:
    wf = wrapper(f,errors='replace')
    count = 0
    page_list = []
    for line in wf:
        count += 1
        info_list = linefilter.findall(line)
        if info_list[0] == 'en.z': # english wikipedia prefix
            page_list.append(info_list[1:])
            #print(line)
            #print(info_list)
        if (count % 100000) == 0:
            print(count)

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000
1

In [42]:
page_list[1]

['!!', '10', 'A2J2K2N1P1W2']

In [104]:
df=pandas.DataFrame(page_list,columns=['Title','DailyTotal','EncodedTimeseries'])
df.DailyTotal = df.DailyTotal.astype(int)

In [45]:
df.to_csv('encoded_timeseries.csv')

In [105]:
df.shape

(6368165, 3)

In [106]:
df.EncodedTimeseries[1]

'A2J2K2N1P1W2'

In [107]:
df.loc[1,'EncodedTimeseries']

'A2J2K2N1P1W2'

In [108]:
df[:10]

,Title,DailyTotal,EncodedTimeseries
0,!,22,A2C1D1E4H1I1K3N2O2R1S1U1W1X1
1,!!,10,A2J2K2N1P1W2
2,!!!,124,A4B7C5D1E6F3G1H3I7J2K6L4M6N3O4P6Q12R7S6T5U8V7W4X7
3,!!!Fuck_You!!!,12,A2E1F1H1J1K2O1R1S1X1
4,!!!Fuck_You!!!_And_Then_Some,3,A1K1X1
5,!!!Fuck_You!!!_and_Then_Some,5,A1L1O1P1S1
6,!!!_(Chk_Chk_Chk),2,A1L1
7,!!!_(album),13,A2D1F1H1I1L2R3X2
8,!!!_(band),2,A1L1
9,!!!_(disambiguation),3,A1L1R1


In [52]:
re.findall('\w\d+','A2J2K2N1P122W2')

['A2', 'J2', 'K2', 'N1', 'P122', 'W2']

In [111]:
def decode_ts(df,visit_threshold):
    # df the dataframe of encoded timeseries
    # visit_threshold : minimal number of visits during the day to be selected
    ts_df = pandas.DataFrame(dtype=int)
    for row in df.itertuples():
        if int(row.DailyTotal) >= visit_threshold:
            row_idx = row.Index
            hourly_sliced = re.findall('\w\d+',row.EncodedTimeseries)
            for hour_slice in hourly_sliced:
                ts_df.loc[row_idx, hour_slice[0]] = int(hour_slice[1:])
    ts_df = ts_df.fillna(0).astype(int)
    ts_df.sort_index(axis=1, inplace=True)
    return ts_df

In [122]:
min_visits_per_day = 1000
df_ts = decode_ts(df,min_visits_per_day)

In [113]:
df_ts

,A,B,C,D,E,F,G,H,I,J,...,O,P,Q,R,S,T,U,V,W,X
765,55,52,64,51,55,41,34,37,27,34,...,52,55,45,70,55,62,49,58,63,72
1980,31,35,48,50,50,48,36,47,73,68,...,64,58,65,66,48,40,37,28,36,45
2786,25750,31672,24585,18424,28896,39840,28245,18861,21942,32797,...,35230,26247,32727,31910,44600,43320,64675,34080,24319,33537
3772,80,54,65,48,71,77,98,120,111,122,...,116,148,110,99,98,97,103,100,61,58
4670,56,41,40,29,57,61,48,48,45,58,...,52,66,80,69,85,96,72,87,106,72
5057,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,9,81,344,604,611,522
7779,51,33,28,43,44,32,35,57,68,48,...,97,74,76,63,75,70,70,83,43,52
8043,37,43,44,46,36,40,41,32,35,54,...,42,50,50,58,56,71,82,68,59,47
10098,55,31,38,32,34,39,56,101,118,153,...,124,116,106,65,58,78,72,70,52,29
11058,51,68,60,48,41,61,36,42,57,55,...,71,79,64,64,65,56,64,71,64,59


In [114]:
result = pandas.merge(df.drop(columns='EncodedTimeseries'), df_ts, how='right', left_index=True, right_index=True)

In [129]:
result = result.sort_values('DailyTotal', ascending=False)

In [130]:
csv_file = 'timeseries-' + date + '-T' + str(min_visits_per_day) + '.csv'
result.to_csv(csv_file,index=False)